In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from cfd_model.cfd_model import load_model, get_generated_dataset
from cfd_model.cfd_dataset import get_dataset
from project_constants import DATASET_FILE, MODEL_FOLDER, WINDOW, DATASET_GENERATED_FILE
from image_functions import preprocess_frame, plot_compare_frames, plot_frame
from plot_functions import plot_frame_histogram

MODEL_FILE = os.path.join(MODEL_FOLDER, "model_20240511_1704.h5")

In [ ]:
dataset, X_train, y_train, X_val, y_val = get_dataset(DATASET_FILE)

In [ ]:
model = load_model(MODEL_FILE)
# model.summary()

In [ ]:
# training_evaluation = model.evaluate(X_train, [X_train, y_train])
# testing_evaluation = model.evaluate(X_val, [X_val, y_val])

In [ ]:
# Visualize Encoder-Decoder
sequence_id = 100
example_sequence = X_train[sequence_id].copy()
# print("example_sequence:", np.min(example_sequence), np.max(example_sequence), np.mean(example_sequence), np.median(example_sequence), np.var(example_sequence))

reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
# print("reconstructed_sequence:", np.min(reconstructed_sequence), np.max(reconstructed_sequence), np.mean(reconstructed_sequence), np.median(reconstructed_sequence), np.var(reconstructed_sequence))

for i in range(len(example_sequence)):
    original = example_sequence[i]
    preprocess_frame(original)
    reconstructed = reconstructed_sequence[i]
    preprocess_frame(reconstructed)
    plot_compare_frames(original, reconstructed, 
                        f"Original {i}", f"Reconstructed {i}",
                        sequence_id, i, 
                        orientation="v")
    plt.show()


In [ ]:
# Visualize Encoded
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()

preprocess_frame(example_frame)
plot_frame(example_frame, title="Original Frame")

# Visualize Reconstructed
example_sequence = X_train[100].copy()

reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
reconstructed_frame = reconstructed_sequence[0]

preprocess_frame(reconstructed_frame)
plot_frame(reconstructed_frame, title="Reconstructed Frame")

In [ ]:
# Reconstructed Histogram
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()
reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
reconstructed_frame = reconstructed_sequence[0]

preprocess_frame(example_frame)
preprocess_frame(reconstructed_frame)

plot_frame_histogram(example_frame, "Histogram Encoded Frame")

plot_frame_histogram(reconstructed_frame, "Histogram Reconstructed Frame")


In [ ]:
# Compare histograms
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]
frame_min=np.min(frame[frame!=-1])
frame_max=np.max(frame)

frame_data = frame.flatten()
predicted_data = predicted_frame.flatten()

frame_data = np.delete(frame_data, np.where(frame_data == -1))

predicted_data = np.delete(predicted_data, np.where(predicted_data<frame_min))
predicted_data = np.delete(predicted_data, np.where(predicted_data>frame_max))


hist_frame, bin_edges_frame = np.histogram(frame_data, bins='auto')
bin_center_frame = (bin_edges_frame[:-1] + bin_edges_frame[1:])/2
hist_predicted, bin_edges_predicted = np.histogram(predicted_data, bins='auto')
bin_center_predicted = (bin_edges_predicted[:-1] + bin_edges_predicted[1:])/2

plt.figure()
plt.figure(figsize=(20,6))
plt.errorbar(bin_center_frame, hist_frame, yerr=0, fmt='.', color="green")
plt.errorbar(bin_center_predicted, hist_predicted, yerr=0, fmt='|', color="blue")
plt.show()

In [ ]:
# Visualize residuals
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]

# residual = np.abs(frame - predicted_frame)
residual = np.square(frame - predicted_frame)

plt.clf()
plt.imshow(np.squeeze(residual).transpose(), cmap="Grays")
plt.colorbar(label="error", orientation="horizontal")

In [ ]:
# Plot original vs generated frames

sequence_id = 19
example_sequence = dataset[sequence_id].copy()
generated_sequence = get_generated_dataset(DATASET_GENERATED_FILE)[sequence_id].copy()
print(example_sequence.shape)
print(generated_sequence.shape)

limit = 5
# limit = len(example_sequence)

for i in range(limit):
    original = example_sequence[i]
    preprocess_frame(original)
    generated = generated_sequence[i]
    preprocess_frame(generated)
    plot_compare_frames(original, generated, 
                        f"Original {i}", f"Generated {i}",
                        sequence_id, i, 
                        orientation="v")
    plt.show()